In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from itertools import product

In [4]:
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('ytick', labelsize=MEDIUM_SIZE)   # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the figure suptitle
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
def plot_training_curves(train_losses, test_losses, train_mees, test_mees, hyperparams):
    """
    Plot training and test curves for loss and Mean Euclidean Error (MEE).

    Parameters:
    - epoch (int): The total number of training epochs.
    - train_losses (list): List of training losses for each epoch.
    - test_losses (list): List of test losses for each epoch.
    - train_mees (list): List of training MEE values for each epoch.
    - test_mees (list): List of test MEE values for each epoch.
    - hyperparams (list): List of hyperparameters used for the plot.

    Returns:
    None

    Plots four subplots:
    1. Training and test loss curves.
    2. Training and test MEE curves.
    3. Zoomed-in training and test loss curves with y-axis limit [0, 10].
    4. Zoomed-in training and test MEE curves with y-axis limit [0, 10].

    The hyperparameters are used in the plot title to provide additional context.
    """

    plt.figure(figsize=(12, 8))
    #plt.suptitle(f'Batch Size={hyperparams[3]},Activation Function={hyperparams[5]}, Layers={hyperparams[6]} Hidden Units={hyperparams[0]}, Eta={hyperparams[1]}, Alpha={hyperparams[2]}, Lambda={hyperparams[4]}, dropout = {hyperparams[7]}')
    # Loss plots
    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Training Loss', color = 'red')
    plt.plot(test_losses, label='Validation Loss', color = 'blue', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.legend()

    # MEE plots
    plt.subplot(2, 2, 2)
    plt.plot(train_mees, label='Training MEE', color='red')
    plt.plot(test_mees, label='Validation MEE', color = 'blue', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('MEE')
    plt.legend()

    # Loss plots
    plt.subplot(2, 2, 3)
    plt.plot(train_losses, label='Training Loss', color = 'red')
    plt.plot(test_losses, label='Validation Loss', color = 'blue', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.ylim(0,5)
    plt.legend()

    # MEE plots
    plt.subplot(2, 2, 4)
    plt.plot(train_mees, label='Training MEE', color='red')
    plt.plot(test_mees, label='Validation MEE', color = 'blue', linestyle='--')
    plt.xlabel('Epoch')
    plt.ylabel('MEE')
    plt.ylim(0,5)
    plt.legend()

    plt.tight_layout()

    plt.show()

In [7]:
def mean_euclidean_error(tensor1, tensor2):
    """
    Compute the mean Euclidean error between two sets of 3D vectors.

    Parameters:
    - tensor1: PyTorch tensor of size (N, 3) representing the first set of 3D vectors
    - tensor2: PyTorch tensor of size (N, 3) representing the second set of 3D vectors

    Returns:
    - mean_error: Mean Euclidean error between the two sets of vectors
    """
    # Check if the tensors have the correct shape
    if tensor1.shape[1] != 3 or tensor2.shape[1] != 3 or tensor1.shape[0] != tensor2.shape[0]:
        raise ValueError("Input tensors must be of size (N, 3)")


    # Compute Euclidean distance
    euclidean_distance = torch.norm(tensor1 - tensor2, dim=1)

    # Calculate the mean Euclidean error
    mean_error = torch.mean(euclidean_distance)

    return mean_error.item()

In [8]:
# Define a regression eural network

class RegressorNN(nn.Module):
    def __init__(self, hidden_sizes, activation_function, input_size=10, output_size=3):
        super(RegressorNN, self).__init__()

        # Input layer
        self.layers = [nn.Linear(input_size, hidden_sizes[0])]

        self.layers.append(nn.Linear(hidden_sizes[0], hidden_sizes[1]))
        self.layers.append(activation_function)
        self.layers.append(nn.Linear(hidden_sizes[1], hidden_sizes[2]))
        self.layers.append(activation_function)

        # Output layer
        self.layers.append(nn.Linear(hidden_sizes[2], output_size))

        # Create a Sequential container for the layers
        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        return self.model(x)

In [9]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

def training_model(x_train, y_train, x_test, y_test, neuron_number,learning_rate, momentum, bs, reg_coeff, activation, optimiz, proportions, num_epochs=1000, plot_curves=False, return_history=False):
    """
    Train the regression model and evaluate it on a test (can also be validation, depending on the context).

    Parameters:
    - x_train (torch.Tensor): Training input data.
    - y_train (torch.Tensor): Training target data.
    - x_test (torch.Tensor): Test input data.
    - y_test (torch.Tensor): Test target data.
    - neuron_number (int): Total number of neurons across all layers.
    - learning_rate (float): Learning rate for the optimizer.
    - momentum (float): Momentum for the optimizer.
    - bs (int): Batch size for training.
    - reg_coeff (float): Regularization coefficient for weight decay.
    - activation (torch.nn.Module): Activation function for the model.
    - layers (int): Number of hidden layers in the model.
    - num_epochs (int, optional): Number of training epochs (default: 1000).
    - plot_curves (bool, optional): Whether to plot training curves (default: False).

    Returns:
    - tuple: Tuple containing the number of epochs, final training loss, final test loss, final training MEE, and final test MEE.

    The function trains a neural network regression model using the specified hyperparameters and evaluates its performance on the test set.
    """
    hidden_sizes = [0,0,0]

    for i in range(len(proportions)):
        hidden_sizes[i] = int(proportions[i]*neuron_number)
        #print(hidden_sizes)
    #print(hidden_size)
    # Create an instance of the model
    model = RegressorNN(hidden_sizes=hidden_sizes, activation_function=activation)
    model.to(device)
    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    if optimiz == 'SGD':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=reg_coeff)
    if optimiz == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg_coeff)
    if optimiz == 'RMS':
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,momentum=momentum, weight_decay=reg_coeff)

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

    train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)

    # Lists to store training and test losses for plotting
    train_losses, test_losses, train_mees, test_mees = [], [], [], []

    # parameters to stop at training convergence
    min_mee = float('inf')
    patience_counter, patience = 0, 20

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        for inputs, labels in train_dataloader:
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels) #Compute the loss

            optimizer.zero_grad()   # Zero the gradients
            loss.backward() # Backward pass
            optimizer.step()  # Update weights

        model.eval()  # Set the model to evaluation mode
        #calculate loss
        y_pred = model(x_train)
        train_loss = criterion(y_pred, y_train)
        # Calculate mee
        train_mee = mean_euclidean_error(y_pred,y_train)
        train_losses.append(train_loss.item())
        train_mees.append(train_mee)

        # Evaluation on the test/validation set
        with torch.no_grad():
            test_outputs = model(x_test)
            test_loss = criterion(test_outputs, y_test)

            # Calculate test mee
            test_mee = mean_euclidean_error(test_outputs,y_test)
            test_mees.append(test_mee)
            test_losses.append(test_loss.item())

        print(f'Training - Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss.item():.4f}, '
            f'MEE: {train_mee:.4f} | Test - Epoch [{epoch+1}/{num_epochs}], '
            f'Loss: {test_loss.item():.4f} MEE: {test_mee:.4f} ', end='\r')

        # Check for convergence
        if train_mee < min_mee and abs(train_mee-min_mee)>1e-3:
            patience_counter = 0
            min_mee = train_mee
        else:
            patience_counter += 1

        if patience_counter == patience:
        #    print(f"Convergence reached. at epoch {epoch+1} Stopping training.")
            break

        # Update the learning rate using the scheduler
        scheduler.step(train_mee)

    print(f'\n N. Epochs = {epoch+1} - Loss (train | test)= ({train_loss.item():.4} | {test_loss.item():.4}) - MEE (train | test) = ({train_mee} | {test_mee})')

    if plot_curves:
        hyperparams = [hidden_sizes,learning_rate, momentum, bs, reg_coeff, activation, proportions, num_epochs]
        plot_training_curves(train_losses, test_losses, train_mees, test_mees, hyperparams)

    if return_history:
        return model, epoch+1, train_loss.item(), test_loss.item(), train_mee, test_mee, np.array(train_losses), np.array(test_losses), np.array(train_mees), np.array(test_mees)
    else:
        return model, epoch+1, train_loss.item(), test_loss.item(), train_mee, test_mee

In [10]:
def reshape_history(N,hist):
    min_len = float('inf')
    for i in range(0,N):
        #print(len(train_losses_hist[i]))
        if len(hist[i]) < min_len:
            min_len = len(hist[i])

    for i in range(0,N):
        hist[i]=hist[i][:min_len]

    return hist
def train_with_different_initializations(x_train, y_train, x_test, y_test,
                                         neuron_number,learning_rate, momentum,
                                         bs, reg_coeff, activation,optimiz,proportions, max_num_epochs=1000,
                                         plot_curves=False, N = 5, return_history=False):
    """
    Train the model multiple times with different weight initializations to estimate performance mean and variance.

    Parameters:
    - x_train (torch.Tensor): Training input data.
    - y_train (torch.Tensor): Training target data.
    - x_test (torch.Tensor): Test input data.
    - y_test (torch.Tensor): Test target data.
    - neuron_number (int): Total number of neurons across all layers.
    - learning_rate (float): Learning rate for the optimizer.
    - momentum (float): Momentum for the optimizer.
    - bs (int): Batch size for training.
    - reg_coeff (float): Regularization coefficient for weight decay.
    - activation (torch.nn.Module): Activation function for the model.
    - layers (int): Number of hidden layers in the model.
    - num_epochs (int, optional): Number of training epochs (default: 1000).
    - plot_curves (bool, optional): Whether to plot training curves (default: False).
    - N (int, optional): Number of times to train the model with different initializations (default: 5).

    Returns:
    None

    Prints the mean and standard deviation of training and test loss, as well as training and test Mean Euclidean Error (MEE).
    """
    final_train_loss = []
    final_test_loss = []
    final_train_mee = []
    final_test_mee = []
    train_losses_hist, test_losses_hist, train_mees_hist, test_mees_hist = [[] for _ in range(N)],  [[] for _ in range(N)],  [[] for _ in range(N)],  [[] for _ in range(N)]
    models = []
    for i in range (0,N):
        model , num_epochs,train_loss, test_loss, train_mee, test_mee, train_losses_hist[i], test_losses_hist[i], train_mees_hist[i], test_mees_hist[i] = training_model(x_train,
                        y_train,
                        x_test,
                        y_test,
                        neuron_number,
                        learning_rate,
                        momentum,
                        bs,
                        reg_coeff,
                        activation,
                        optimiz,
                        proportions,
                        plot_curves=plot_curves,
                        num_epochs=max_num_epochs,
                        return_history=True)

        final_train_loss.append(train_loss)
        final_test_loss.append(test_loss)
        final_train_mee.append(train_mee)
        final_test_mee.append(test_mee)
        models.append(model)

    if plot_curves:
        hyperparams = [learning_rate, momentum, bs, reg_coeff, activation, num_epochs]
        plot_training_curves(train_losses=train_losses_hist[i],test_losses=test_losses_hist[i],train_mees=train_mees_hist[i],test_mees=test_mees_hist[i],hyperparams=hyperparams)

    plt.show()
    print(f'Avg of {N} initializations: Loss (train | test/val )= ({np.mean(final_train_loss):.4} +- {np.std(final_train_loss):.4} | {np.mean(final_test_loss):.4} +- {np.std(final_test_loss):.4})'
          f'- MEE (train | test/val ) = ( {np.mean(final_train_mee):.4} +-  {np.std(final_train_mee):.4} | {np.mean(final_test_mee):.4} +- {np.std(final_test_mee):.4})')


    if return_history:
        train_losses_hist = reshape_history(N,train_losses_hist)
        test_losses_hist = reshape_history(N,test_losses_hist)
        train_mees_hist = reshape_history(N,train_mees_hist)
        test_mees_hist = reshape_history(N,test_mees_hist)
        return models, train_losses_hist, test_losses_hist, train_mees_hist, test_mees_hist
    else:
        return np.mean(final_train_loss), np.mean(final_test_loss), np.mean(final_train_mee), np.mean(final_test_mee)

In [11]:
def perform_grid_search_kfold(neuron_numbers, learning_rates, momentums, batch_sizes, reg_coeffs, activations,optimiz,proportionss, k_folds, x, y,N=3, plot_curves=False, num_epochs=1000):
    """
    Perform grid search with k-fold cross-validation for hyperparameters.

    Parameters:
    - neuron_numbers (list): List of neuron numbers to search.
    - learning_rates (list): List of learning rates to search.
    - momentums (list): List of momentum values to search.
    - batch_sizes (list): List of batch sizes to search.
    - reg_coeffs (list): List of regularization coefficients to search.
    - activations (list): List of activation functions to search.
    - layerss (list): List of numbers of hidden layers to search.
    - k_folds (int): Number of folds for cross-validation.
    - x (numpy.ndarray): Input data.
    - y (numpy.ndarray): Target data.
    - plot_curves (bool, optional): Whether to plot training curves (default: False).
    - num_epochs (int, optional): Number of training epochs (default: 1000).

    Returns:
    - list: List of best hyperparameters.

    The function performs grid search with k-fold cross-validation for Monk classifier hyperparameters and returns the best hyperparameters.
    """

    best_mee = float('inf')
    best_hyperparams = []

    for neuron_number, learning_rate, momentum, bs, reg_coeff, activation, proportions in product(neuron_numbers,learning_rates,momentums,batch_sizes, reg_coeffs, activations, proportionss):
        print(f'activation={activation};; neuron_number={neuron_number}; lr={learning_rate}; alpha = {momentum}; batch size = {bs}; lambda = {reg_coeff}; optim = {optimiz}; proportions = {proportions}')

        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

        # Lists to store training and validation losses and MEEs for each epoch
        train_losses, val_losses, train_mees, val_mees = [], [], [], []

        # Perform K-fold cross-validation
        for fold, (train_indices, val_indices) in enumerate(kf.split(x,y)):
            #print(f"\nFold {fold + 1}/{k_folds}")

            # Split the data into training and validation (or test) sets
            X_train, X_val = x[train_indices], x[val_indices]
            Y_train, Y_val = y[train_indices], y[val_indices]

            train_loss, val_loss, train_mee, val_mee = train_with_different_initializations(X_train, Y_train, X_val, Y_val,
                neuron_number=neuron_number, learning_rate=learning_rate, momentum=momentum,
                bs=bs, reg_coeff=reg_coeff, activation=activation, proportions=proportions, optimiz=optimiz ,plot_curves=plot_curves, max_num_epochs=num_epochs, N=N)

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_mees.append(train_mee)
            val_mees.append(val_mee)

        print(f'Final Results: activation={activation}; neuron number={neuron_number}; lr={learning_rate}; alpha = {momentum}; batch size = {bs}; lambda = {reg_coeff}; proportions = {proportions} --> '
            f'train_loss = {np.mean(train_losses):.4} +- {np.std(train_losses):.4} | '
            f'val_loss = {np.mean(val_losses):.4} +- {np.std(val_losses):.4}'
            f'train_mee = {np.mean(train_mees):.4} +- {np.std(train_mees):.4} | '
            f'val_mee = {np.mean(val_mees):.4} +- {np.std(val_mees):.4}')

        if np.mean(val_mees) < best_mee:
            best_mee = np.mean(val_mees)
            best_hyperparams = [neuron_number, learning_rate, momentum, bs, reg_coeff, activation, proportions]

    print(best_hyperparams)
    return best_hyperparams

In [12]:
def plot_mean_std(N,train_hist, test_hist,label):
    mean_tr = np.array(train_hist).mean(axis=0)
    std_tr = np.array(train_hist).std(axis=0)
    mean_te = np.array(test_hist).mean(axis=0)
    std_te = np.array(test_hist).std(axis=0)

    plt.figure(figsize=(9, 8))
    plt.subplot(2,1,1)
    plt.plot(mean_tr, label=f'Training {label} (mean $\pm$ std)', color = 'red', linewidth=1)
    plt.fill_between(range(0,len(train_hist[0])),mean_tr-std_tr, mean_tr+std_tr, color='crimson', alpha=0.3)

    plt.plot(mean_te, label=f'Test {label} (mean $\pm$ std)', color = 'blue', linestyle='--', linewidth=1)
    plt.fill_between(range(0,len(test_hist[0])),mean_te-std_te, mean_te+std_te, color='blue', alpha=0.3)

    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.legend()

    plt.subplot(2,1,2)

    plt.plot(mean_tr, label=f'Training {label} (mean $\pm$ std)', color = 'red', linewidth=1)
    plt.fill_between(range(0,len(train_hist[0])),mean_tr-std_tr, mean_tr+std_tr, color='crimson', alpha=0.3)

    plt.plot(mean_te, label=f'Test {label} (mean $\pm$ std)', color = 'blue', linestyle='--', linewidth=1)
    plt.fill_between(range(0,len(test_hist[0])),mean_te-std_te, mean_te+std_te, color='blue', alpha=0.3)

    plt.xlabel('Epoch')
    plt.ylabel(label)
    plt.ylim(0,5)
    plt.legend()
    plt.show()


# TRAIN THE MODEL USING DIFFERENT INITIALIZATIONS

# IMPORT THE DATASET

In [13]:
from sklearn.model_selection import train_test_split
# load the dataset, split into input (X) and output (y) variables
dataset = np.loadtxt('ML-CUP23-TR.csv', delimiter=',')
X = dataset[:,1:11]
y = dataset[:,11:14]

X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

X = X.to(device)
y = y.to(device)

# Split the data into training and testing sets (80%/20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# PERFORM GRID SEARCH TO FIND BEST HYPERPARAMETERS

In [14]:
hidden_neurons = [1000] #total number of neurons
learning_rates = [1e-5]
momentums = [0.9] #if optimiz = 'Adam' it doesn't matter
batch_sizes = [128]
reg_coeffs = [1e-3]
activations = [nn.Tanh()]
proportionss = [[0.1,0.8,0.1],[0.2,0.6,0.2],[0.25,0.50,0.25],[0.2,0.7,0.1],[0.1,0.7,0.2],[0.3,0.6,0.1]]
#proportionss = [[0.8,0.1,0.1],[0.7,0.2,0.1],[0.6,0.3,0.1],[0.6,0.2,0.2],[0.8,0.15,0.05],[0.5,0.4,0.1],[0.5,0.25,0.25], [0.5,0.3,0.2]]
#proportionss = [[0.1,0.1,0.8],[0.1,0.2,0.7],[0.1,0.3,0.6],[0.2,0.2,0.6],[0.05,0.15,0.8],[0.1,0.4,0.5],[0.25,0.25,0.5], [0.2,0.3,0.5]]
#proportionss = [[0.2,0.3,0.5],[0.25,0.25,0.5],[0.15,0.35,0.5],[0.15,0.3,0.55],[0.1,0.25,0.55],[0.05,0.40,0.55],[0.25,0.35,0.4]]
optimiz = 'RMS'
best_hp = perform_grid_search_kfold(hidden_neurons,
                                    learning_rates,
                                    momentums,
                                    batch_sizes,
                                    reg_coeffs,
                                    activations,
                                    optimiz,
                                    proportionss,
                                    k_folds=3,
                                    x=X_train,
                                    y=y_train,
                                    num_epochs=5000,
                                    plot_curves=False,
                                    N=3)

activation=Tanh();; neuron_number=1000; lr=1e-05; alpha = 0.9; batch size = 128; lambda = 0.001; optim = RMS; proportions = [0.1, 0.8, 0.1]


Training - Epoch [2065/5000], Loss: 1.1543, MEE: 0.8375 | Test - Epoch [2065/5000], Loss: 0.7918 MEE: 1.1040     
 N. Epochs = 2065 - Loss (train | test)= (1.154 | 0.7918) - MEE (train | test) = (0.8374984264373779 | 1.1039910316467285)
Epoch 02325: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [2325/5000], Loss: 0.4866 MEE: 0.8448     
Epoch 02501: reducing learning rate of group 0 to 2.5000e-06.t - Epoch [2501/5000], Loss: 0.4630 MEE: 0.8284 
Training - Epoch [2613/5000], Loss: 0.0699, MEE: 0.3328 | Test - Epoch [2613/5000], Loss: 0.4461 MEE: 0.7970 
 N. Epochs = 2613 - Loss (train | test)= (0.06993 | 0.4461) - MEE (train | test) = (0.33284276723861694 | 0.7970279455184937)
Epoch 02391: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [2391/5000], Loss: 0.5683 MEE: 0.8543     
Epoch 02435: reducing learning rate of group 0 to 2.5000e-06.t - Epoch [2435/5000], Loss: 0.5463 MEE: 0.8370 
Epoch 02471: reducing learning rate of group 0 to 1.2500e-06.t - Epoch [2471/50

In [15]:
hidden_neurons = [1000] #total number of neurons
learning_rates = [1e-5]
momentums = [0.9] #if optimiz = 'Adam' it doesn't matter
batch_sizes = [128]
reg_coeffs = [1e-3]
activations = [nn.Tanh()]
#proportionss = [[0.1,0.8,0.1],[0.2,0.6,0.2],[0.25,0.50,0.25],[0.2,0.7,0.1],[0.1,0.7,0.2],[0.3,0.6,0.1]]
proportionss = [[0.8,0.1,0.1],[0.7,0.2,0.1],[0.6,0.3,0.1],[0.6,0.2,0.2],[0.8,0.15,0.05],[0.5,0.4,0.1],[0.5,0.25,0.25], [0.5,0.3,0.2]]
#proportionss = [[0.1,0.1,0.8],[0.1,0.2,0.7],[0.1,0.3,0.6],[0.2,0.2,0.6],[0.05,0.15,0.8],[0.1,0.4,0.5],[0.25,0.25,0.5], [0.2,0.3,0.5]]
#proportionss = [[0.2,0.3,0.5],[0.25,0.25,0.5],[0.15,0.35,0.5],[0.15,0.3,0.55],[0.1,0.25,0.55],[0.05,0.40,0.55],[0.25,0.35,0.4]]
optimiz = 'RMS'
best_hp = perform_grid_search_kfold(hidden_neurons,
                                    learning_rates,
                                    momentums,
                                    batch_sizes,
                                    reg_coeffs,
                                    activations,
                                    optimiz,
                                    proportionss,
                                    k_folds=3,
                                    x=X_train,
                                    y=y_train,
                                    num_epochs=5000,
                                    plot_curves=False,
                                    N=3)

activation=Tanh();; neuron_number=1000; lr=1e-05; alpha = 0.9; batch size = 128; lambda = 0.001; optim = RMS; proportions = [0.8, 0.1, 0.1]
Epoch 02034: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [2034/5000], Loss: 0.8040 MEE: 1.0254     
Epoch 02216: reducing learning rate of group 0 to 2.5000e-06.t - Epoch [2216/5000], Loss: 0.7480 MEE: 0.9625 
Epoch 02339: reducing learning rate of group 0 to 1.2500e-06.t - Epoch [2339/5000], Loss: 0.7365 MEE: 0.9429 
Training - Epoch [2400/5000], Loss: 0.0785, MEE: 0.3508 | Test - Epoch [2400/5000], Loss: 0.7396 MEE: 0.9433 
 N. Epochs = 2400 - Loss (train | test)= (0.07854 | 0.7396) - MEE (train | test) = (0.35078898072242737 | 0.9432961940765381)
Epoch 02187: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [2187/5000], Loss: 1.0519 MEE: 1.1063     
Epoch 02304: reducing learning rate of group 0 to 2.5000e-06.t - Epoch [2304/5000], Loss: 1.0152 MEE: 1.0729 
Training - Epoch [2447/5000], Loss: 0.0517, MEE: 0.2966 | Test - E

In [16]:
hidden_neurons = [1000] #total number of neurons
learning_rates = [1e-5]
momentums = [0.9] #if optimiz = 'Adam' it doesn't matter
batch_sizes = [128]
reg_coeffs = [1e-3]
activations = [nn.Tanh()]
#proportionss = [[0.1,0.8,0.1],[0.2,0.6,0.2],[0.25,0.50,0.25],[0.2,0.7,0.1],[0.1,0.7,0.2],[0.3,0.6,0.1]]
#proportionss = [[0.8,0.1,0.1],[0.7,0.2,0.1],[0.6,0.3,0.1],[0.6,0.2,0.2],[0.8,0.15,0.05],[0.5,0.4,0.1],[0.5,0.25,0.25], [0.5,0.3,0.2]]
proportionss = [[0.1,0.1,0.8],[0.1,0.2,0.7],[0.1,0.3,0.6],[0.2,0.2,0.6],[0.05,0.15,0.8],[0.1,0.4,0.5],[0.25,0.25,0.5], [0.2,0.3,0.5]]
#proportionss = [[0.2,0.3,0.5],[0.25,0.25,0.5],[0.15,0.35,0.5],[0.15,0.3,0.55],[0.1,0.25,0.55],[0.05,0.40,0.55],[0.25,0.35,0.4]]
optimiz = 'RMS'
best_hp = perform_grid_search_kfold(hidden_neurons,
                                    learning_rates,
                                    momentums,
                                    batch_sizes,
                                    reg_coeffs,
                                    activations,
                                    optimiz,
                                    proportionss,
                                    k_folds=3,
                                    x=X_train,
                                    y=y_train,
                                    num_epochs=5000,
                                    plot_curves=False,
                                    N=3)

activation=Tanh();; neuron_number=1000; lr=1e-05; alpha = 0.9; batch size = 128; lambda = 0.001; optim = RMS; proportions = [0.1, 0.1, 0.8]
Epoch 01110: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [1110/5000], Loss: 0.4811 MEE: 0.9321   
Training - Epoch [1224/5000], Loss: 0.2282, MEE: 0.7180 | Test - Epoch [1224/5000], Loss: 0.4347 MEE: 0.8640 
 N. Epochs = 1224 - Loss (train | test)= (0.2282 | 0.4347) - MEE (train | test) = (0.7179601788520813 | 0.8640018701553345)
Epoch 01341: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [1341/5000], Loss: 0.4688 MEE: 0.8862   
Training - Epoch [1644/5000], Loss: 0.1611, MEE: 0.6056 | Test - Epoch [1644/5000], Loss: 0.4057 MEE: 0.8139 
 N. Epochs = 1644 - Loss (train | test)= (0.1611 | 0.4057) - MEE (train | test) = (0.605635941028595 | 0.8138821721076965)
Epoch 01191: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [1191/5000], Loss: 0.4094 MEE: 0.8389   
Training - Epoch [1282/5000], Loss: 0.2024, MEE: 0.6764 |

In [17]:
hidden_neurons = [1000] #total number of neurons
learning_rates = [1e-5]
momentums = [0.9] #if optimiz = 'Adam' it doesn't matter
batch_sizes = [128]
reg_coeffs = [1e-3]
activations = [nn.Tanh()]
#proportionss = [[0.1,0.8,0.1],[0.2,0.6,0.2],[0.25,0.50,0.25],[0.2,0.7,0.1],[0.1,0.7,0.2],[0.3,0.6,0.1]]
#proportionss = [[0.8,0.1,0.1],[0.7,0.2,0.1],[0.6,0.3,0.1],[0.6,0.2,0.2],[0.8,0.15,0.05],[0.5,0.4,0.1],[0.5,0.25,0.25], [0.5,0.3,0.2]]
#proportionss = [[0.1,0.1,0.8],[0.1,0.2,0.7],[0.1,0.3,0.6],[0.2,0.2,0.6],[0.05,0.15,0.8],[0.1,0.4,0.5],[0.25,0.25,0.5], [0.2,0.3,0.5]]
proportionss = [[0.2,0.3,0.5],[0.25,0.25,0.5],[0.15,0.35,0.5],[0.15,0.3,0.55],[0.1,0.25,0.55],[0.05,0.40,0.55],[0.25,0.35,0.4]]
optimiz = 'RMS'
best_hp = perform_grid_search_kfold(hidden_neurons,
                                    learning_rates,
                                    momentums,
                                    batch_sizes,
                                    reg_coeffs,
                                    activations,
                                    optimiz,
                                    proportionss,
                                    k_folds=3,
                                    x=X_train,
                                    y=y_train,
                                    num_epochs=5000,
                                    plot_curves=False,
                                    N=3)

activation=Tanh();; neuron_number=1000; lr=1e-05; alpha = 0.9; batch size = 128; lambda = 0.001; optim = RMS; proportions = [0.2, 0.3, 0.5]
Epoch 01147: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [1147/5000], Loss: 0.2453 MEE: 0.5766     
Epoch 01213: reducing learning rate of group 0 to 2.5000e-06.t - Epoch [1213/5000], Loss: 0.2356 MEE: 0.5576 
Training - Epoch [1253/5000], Loss: 0.0339, MEE: 0.2825 | Test - Epoch [1253/5000], Loss: 0.2341 MEE: 0.5553 
 N. Epochs = 1253 - Loss (train | test)= (0.03392 | 0.2341) - MEE (train | test) = (0.28251415491104126 | 0.5553089380264282)
Epoch 01079: reducing learning rate of group 0 to 5.0000e-06.t - Epoch [1079/5000], Loss: 0.2413 MEE: 0.5913     
Training - Epoch [1140/5000], Loss: 0.0435, MEE: 0.3185 | Test - Epoch [1140/5000], Loss: 0.2296 MEE: 0.5736 
 N. Epochs = 1140 - Loss (train | test)= (0.04352 | 0.2296) - MEE (train | test) = (0.318450391292572 | 0.5736222267150879)
Epoch 00946: reducing learning rate of group 0 to 5.